In [31]:
import os
from os.path import join
import sys
import cv2 as cv
import numpy as np
from numpy.random import RandomState
import pickle
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [32]:
def load_labels(specific_video=None):
    """ Loads in image data as numpy arrays """
    sequence = []
    none_count = 0 
    filedir = join(os.getcwd(),"labels")
    for file in os.listdir(filedir):
        ## change current seq when video_id change or marker number changes
        if file.endswith(".jpg"):
            file = file.split(".")[0]
            file = file.split("_")
            if specific_video == None:
                video_id, marker_num, marker_type, frame_num, x_pos, y_pos = file[0], int(file[1]), int(file[2]), int(file[3]), int(file[4]), int(file[5])
                current_seq = [video_id, marker_num, marker_type, frame_num, x_pos, y_pos]
                sequence.append(current_seq)
            else:
                if file[0] == video_id:
                    video_id, marker_num, marker_type, frame_num, x_pos, y_pos = file[0], int(file[1]), int(file[2]), int(file[3]), int(file[4]), int(file[5])
                    current_seq = [video_id, marker_num, marker_type, frame_num, x_pos, y_pos]#, dist]
                    sequence.append(current_seq)
    return sequence

In [33]:
def sort_labels(sequence):
    sequence.sort(key=lambda x: x[3]) ## Sort by frame number
    sequence.sort(key=lambda x: x[1]) ## Sort by marker_num
    sequence.sort(key=lambda x: x[0]) ## Sort by video_name
    return sequence

In [54]:
def assign_labels(sequence):
    ##  ball change or vid change
    prev_vid_id, prev_marker_num = sequence[0][0][0], sequence[0][0][1]
    for idx, seq in enumerate(sequence):
        vid_id, marker_num = seq[0], seq[1]
        if (vid_id != prev_vid_id) or (marker_num != prev_marker_num):
            prev_vid_id, prev_marker_num = vid_id, marker_num
            prev_coords, prev_frame_num = np.array([-1,-1]), -1
        frame_num, x_pos, y_pos = seq[3], seq[4], seq[5]
        current_coords = np.array([x_pos, y_pos])
        if (prev_coords[0] == -1) & (prev_coords[1] == -1):
            dist = -1
        else:
            dist = np.linalg.norm(current_coords - prev_coords)
        if (prev_frame_num == -1):
            frame_diff = -1
        else:
            frame_diff = frame_num - prev_frame_num
        prev_coords = current_coords
        prev_frame_num = frame_num
        sequence[idx].append(dist)
        sequence[idx].append(frame_diff)
    return sequence

In [55]:
def load_video(video_path, video_name, flip, display = False): ## Convert 3rd element in video name into flip
    width, height = 960, 540
    cap = cv.VideoCapture(join(video_path,video_name))
    ret, frame = cap.read()
    if (flip):
        frame = cv.flip(frame, 0)
    clone = cv.resize(frame, (width,height))
    if (display):
        cv.namedWindow("Video")
    frame_num = 0
    frames = []
    while (ret):
        if (display):
            cv.imshow("Video", clone)
        frames.append( [frame, frame_num] )
        if (display):
            key = cv.waitKey(0)
            if key == 113:
                break
        ret, frame = cap.read()
        if (ret):
            frame_num += 1
            if (flip):
                frame = cv.flip(frame, 0)
            clone = cv.resize(frame, (width, height))
    cap.release()
    if (display):
        cv.destroyAllWindows()
    return frames

In [56]:
def load_data_and_labels(sequence, vid_format=".avi"):
    filedir = join(os.getcwd(),"resources")
    video_recorded = []
    for file in os.listdir(filedir):
        video_id = file.split(".")[0]
        video_recorded.append(video_id)
    video_annotated = list(sorted(set([i[0] for i in sequence])))
    video_data = []
    """ Checking the videos annotated is in the video recorded """
    for rec in video_recorded:
        if rec in video_annotated:
            data = []
            print("Found: {}".format(rec))
            labels = [i for i in seq if i[0] == rec]
            labels.sort(key=lambda x: x[3]) ## Sort by frame number
            labels = np.asarray(labels)[:,1:].astype('float32')
            frames = load_video(filedir, rec + vid_format, False)
            prev_frame = 0
            frame_labels = []
            for idx, label in enumerate(labels):
                curr_frame = int(label[2])
                if (curr_frame != prev_frame):
                    data.append([frames[prev_frame][0], frame_labels])                    
                    frame_labels = [label]
                elif idx == len(labels) - 1:
                    frame_labels.append(label)
                    data.append([frames[curr_frame][0],frame_labels])
                else:
                    frame_labels.append(label)
                prev_frame = int(curr_frame)
            video_data.append(data)
    return video_data    

In [57]:
seq = load_labels()
seq = sort_labels(seq)
seq = assign_labels(seq)
data = load_data_and_labels(seq)

Found: v0-0
Found: v0-1


In [58]:
data[0]

[[array([[[189, 190, 186],
          [189, 190, 186],
          [189, 190, 186],
          ...,
          [165, 165, 165],
          [165, 165, 165],
          [165, 165, 165]],
  
         [[189, 190, 186],
          [189, 190, 186],
          [189, 190, 186],
          ...,
          [165, 165, 165],
          [165, 165, 165],
          [165, 165, 165]],
  
         [[189, 190, 186],
          [189, 190, 186],
          [189, 190, 186],
          ...,
          [165, 165, 165],
          [165, 165, 165],
          [165, 165, 165]],
  
         ...,
  
         [[ 76,  63,  49],
          [ 75,  62,  48],
          [ 75,  62,  48],
          ...,
          [ 59,  53,  46],
          [ 59,  53,  46],
          [ 59,  53,  46]],
  
         [[ 80,  67,  53],
          [ 78,  65,  51],
          [ 76,  63,  49],
          ...,
          [ 56,  50,  43],
          [ 56,  50,  43],
          [ 56,  50,  43]],
  
         [[ 83,  70,  56],
          [ 80,  67,  53],
          [ 78,  65,  51